# Noun Semantics in the Hebrew Bible

*This notebook will contain the descriptions and analysis for the presentation on noun semantics in the Hebrew Bible.*

In [2]:
# ETCBC's BHSA data
from tf.fabric import Fabric
from tf.extra.bhsa import Bhsa

# stats & data-containers
import collections
import pandas as pd
import numpy as np
import scipy.stats as stats

# data visualizations
import seaborn as sns
sns.set(style="whitegrid")
import matplotlib.pyplot as plt

# prep the data
name = 'noun_semantics'
hebrew_data = ['~/github/etcbc/{}/tf/c'.format(direc) for direc in ('bhsa','lingo/heads', 'phono')] # data dirs
load_features = '''
heads typ phono lex_utf8 freq_lex
''' 

# TF load statements
TF = Fabric(locations=hebrew_data, silent=True)
api = TF.load(load_features)
B = Bhsa(api=api, hoist=globals(), silent=True, name=name) # Bhsa functions for search and visualizing text

  0.00s loading features ...
   |     0.16s B lex_utf8             from /Users/cody/github/etcbc/bhsa/tf/c
   |     0.20s B phono                from /Users/cody/github/etcbc/phono/tf/c
   |     0.85s B heads                from /Users/cody/github/etcbc/lingo/heads/tf/c
   |     0.24s B typ                  from /Users/cody/github/etcbc/bhsa/tf/c
   |     0.60s T freq_lex             from /Users/cody/github/etcbc/bhsa/tf/c
  7.17s All features loaded/computed - for details use loadLog()


## Define a Target Noun Set

*Insert discussion about the semantic relationship between iconicity and frequency with regards to the most frequent noun lexemes in the HB.*

*Also some discussion about Zipfian curves.

In [4]:
search_set = '''

lex sp=subs

'''

noun_occurrences = B.search(search_set)

  0.02s 4076 results


**TODO: Plot all 4076 nouns against their frequency. We should see a Zipfian curve. We can then determine a cut-off point based on the curve from which to derive a set of target nouns which we will describe.** 